In [ ]:
# Fractional Delay Line

from scipy import signal
from scipy.fftpack import fft, ifft
import numpy as np

from IPython.display import Audio 
import soundfile as sf


import numpy as np

class DelayLine:
    """
    A delay line with optional linear interpolation for fractional sample delays.
    """
    def __init__(self, max_delay_samples: int):
        """
        :param max_delay_samples: The allocated maximum delay in samples.
        """
        self.size = max_delay_samples
        self.buffer = np.zeros(self.size, dtype=np.float32)
        self.write_idx = 0

    def write(self, x: float):
        """
        Write one sample to the delay line.
        """
        self.buffer[self.write_idx] = x
        self.write_idx = (self.write_idx + 1) % self.size

    def read(self, delay_samples: float) -> float:
        """
        Return the sample 'delay_samples' behind the write pointer.
        Uses simple linear interpolation for fractional values.
        """
        # Integer part:
        int_delay = int(np.floor(delay_samples))
        frac = delay_samples - int_delay  # fractional part

        # read index for integer portion
        idx1 = self.write_idx - int_delay
        if idx1 < 0:
            idx1 += self.size
        
        # read index for next sample
        idx2 = idx1 - 1
        if idx2 < 0:
            idx2 += self.size

        # if no fractional delay is needed, just return buffer[idx1]
        if frac == 0.0:
            return self.buffer[idx1]

        # otherwise do linear interpolation
        s1 = self.buffer[idx1]
        s2 = self.buffer[idx2]
        return (1.0 - frac) * s1 + frac * s2

    def clear(self):
        self.buffer[:] = 0
        self.write_idx = 0

class OnePoleLowpass:
    """
    y[n] = b*x[n] + (1-b)*y[n-1]
    where b is in (0, 1) -> bigger b = higher cutoff
    """
    def __init__(self, b: float):
        self.b = b
        self.z1 = 0.0  # filter state

    def set_b(self, new_b: float):
        """
        Adjust the lowpass coefficient in real time if desired.
        """
        self.b = new_b

    def process_sample(self, x: float) -> float:
        y = self.b * x + (1.0 - self.b) * self.z1
        self.z1 = y
        return y

    def clear(self):
        self.z1 = 0.0

import math

class ModulatedAllpass:
    """
    A modulated allpass filter with a moveable delay time (fractional).
    """
    def __init__(self, max_delay_samples: int, g: float, sample_rate: float, lfo_rate: float=0.5, lfo_depth: float=5.0):
        """
        :param max_delay_samples: large enough to accommodate your highest delay in samples
        :param g: Allpass feedback/feedforward gain
        :param sample_rate: for the LFO
        :param lfo_rate: frequency in Hz for the delay modulation
        :param lfo_depth: peak +/- amplitude in samples for the delay shift
        """
        self.g = g
        self.delay_line = DelayLine(max_delay_samples)
        self.sample_rate = sample_rate
        self.lfo_rate = lfo_rate
        self.lfo_depth = lfo_depth

        # base delay in samples - set it to something in your constructor, or use set_base_delay()
        self.base_delay = 100.0

        # LFO phase
        self.lfo_phase = 0.0

    def set_base_delay(self, delay_samples: float):
        self.base_delay = delay_samples

    def set_lfo_params(self, rate_hz: float, depth_samples: float):
        """
        Adjust LFO on the fly if desired.
        """
        self.lfo_rate = rate_hz
        self.lfo_depth = depth_samples

    def process_sample(self, x: float) -> float:
        """
        Implementation of the allpass difference equations using the 
        'canonical' approach that stores the 'u[n]' inside the delay line.
        """
        # Step 1) read from delay line:
        #        the allpass is storing u[n-1], so we read with some modulated delay
        current_lfo_offset = self.lfo_depth * math.sin(2 * math.pi * self.lfo_phase)
        delay_samps = self.base_delay + current_lfo_offset
        
        d = self.delay_line.read(delay_samps)

        # Step 2) compute u[n]
        u = x + self.g * d

        # Step 3) output
        y = d - self.g * u

        # Step 4) write the current u[n] to the delay
        self.delay_line.write(u)

        # step LFO phase
        self.lfo_phase += self.lfo_rate / self.sample_rate
        if self.lfo_phase >= 1.0:
            self.lfo_phase -= 1.0

        return y

    def clear(self):
        self.delay_line.clear()
        self.lfo_phase = 0.0

